In [ ]:
# Imports
from pathlib import Path
import json
import pandas as pd
from src.training.trainer import train_ml
from src.training.inference import ModelWrapper
from src.training.adapter import inference_backtest

# Simple simulator that implements the expected interface used by run_backtest
class SimpleSim:
    def __init__(self):
        self.stats = {}
    def place_order(self, order, market_price, is_maker=False):
        return {
            'filled_size': float(order.size),
            'avg_fill_price': float(market_price),
            'fee': 0.0
        }

In [ ]:
# 1) Run the training pipeline (ML branch) to create a checkpoint and metrics
out_dir = Path('examples/artifacts')
out_dir.mkdir(parents=True, exist_ok=True)
ckpt_path = out_dir / 'ml_example_ckpt.pkl'
train_ml(data_root='', save=str(ckpt_path), steps=50, seed=42)
print('Saved checkpoint:', ckpt_path)
print('Metrics file:', str(ckpt_path).replace('.pkl', '.metrics.json'))

In [ ]:
# 2) Load checkpoint and inspect metrics
ckpt = ModelWrapper.from_file(str(ckpt_path))
print('Checkpoint metadata:', {k: ckpt.ckpt.get(k) for k in ('type','seed','steps')})
print('Metrics:', ckpt.ckpt.get('metrics'))

In [ ]:
# 3) Prepare a small synthetic prices DataFrame and compute targets
idx = pd.date_range('2020-01-01', periods=120, freq='T')
import numpy as np
rng = np.random.default_rng(0)
returns = rng.normal(scale=0.001, size=len(idx))
price = 100.0 + np.cumsum(returns)
prices = pd.DataFrame({'close': price}, index=idx)

preds = ckpt.predict_returns(prices)
targets = ckpt.predicted_to_targets(prices, method='vol_norm', vol_window=20, scale=1.0, max_size=2.0)
print('Predictions head:')
print(preds.head())
print('Targets head:')
print(targets.head())

In [ ]:
# 4) Run a backtest using the adapter and the simple simulator
sim = SimpleSim()
result = inference_backtest(str(ckpt_path), prices, sim, method='vol_norm', scale=1.0, vol_window=20, max_size=2.0)
print('Backtest PnL head:')
print(result['pnl'].head())
print('Executions:')
print(result['executions'].head())
print('Stats:')
print(result['stats'])

## Demonstration: Running the CLI `eval` command with a historical CSV

The cell below writes the `prices` DataFrame to a CSV with `timestamp` and `close` columns, then calls the CLI:\n
`python -m orchestration.cli eval --ckpt <ckpt> --prices-csv <csv> --out <outdir>`\n
to run an end-to-end evaluation using the historical file. The CLI will produce a summary JSON in the `--out` folder.

In [ ]:
# Write prices to CSV and run the CLI eval command using that CSV
csv_path = out_dir / 'prices_example.csv'
prices_reset = prices.reset_index()
# ensure timestamp column name is 'timestamp'
prices_reset.rename(columns={prices_reset.columns[0]: 'timestamp'}, inplace=True)
prices_reset[['timestamp','close']].to_csv(csv_path, index=False)
print('Wrote CSV to:', csv_path)

# Run the CLI as a subprocess so the notebook demonstrates the external entrypoint
import subprocess, sys
cmd = [sys.executable, '-m', 'orchestration.cli', 'eval', '--ckpt', str(ckpt_path), '--prices-csv', str(csv_path), '--out', str(out_dir / 'from_csv_eval')]
print('Running:', ' '.join(cmd))
proc = subprocess.run(cmd, capture_output=True, text=True)
print('CLI stdout:
', proc.stdout)
print('CLI stderr:
', proc.stderr)
print('Summary JSON written to:', out_dir / 'from_csv_eval' / 'eval_summary.json')